In [1]:
using GamsStructure

using JuMP.Containers

In [2]:
struct GP{N}
    universe
    sets::Tuple{Vararg{Symbol}}
    value::Array{Float64,N}
    description::String
    GP(GU,sets::Tuple{Vararg{Symbol}},description::String) = new{length(sets)}(GU,sets,zeros(Float64,Tuple(length(GU[e]) for e∈sets)),description)
    GP(GU,sets::Tuple{Vararg{Symbol}};description = "") = new{length(sets)}(GU,sets,zeros(Float64,Tuple(length(GU[e]) for e∈sets)),description)
end


function Base.show(io::IO,P::GP)
    print("Description: $(P.description)\nDomain: $(P.sets)\n\n")
    show(P.value)
end

In [3]:
function convert_idx(P::GP,i::Int,idx::Union{Symbol,Colon})
    GU = P.universe
    set = P.sets[i]
    return collect(1:length(GU[set]))
end

function convert_idx(P::GP,i::Int,idx::Vector{Symbol})
    GU = P.universe
    set = indexin(idx,[e for e in GU[P.sets[i]]])#findall(x-> x∈idx, GU[P.sets[i]])
    if length(set) == 1
        return set[1]
    end
    return set
end

function convert_idx(P::GP,i::Int,idx::Vector{Bool})
    #GU = P.universe
    #set = P.sets[i]
    return idx
end

function convert_idx(P::GP,i::Int,idx)
    return idx
end

function Base.getindex(P::GP,idx...)
    idx = map(x->convert_idx(P,x[1],x[2]),enumerate(idx))
    return P.value[idx...]
end


function Base.iterate(iter::GP)
    next = iterate(iter.value)
    return next === nothing ? nothing : (next[1], next[2])
end

function Base.iterate(iter::GP, state)
    next = iterate(iter.value, state)
    return next === nothing ? nothing : (next[1], next[2])
end


function Base.setindex!(X::GP,value,idx...)
    new_index = map(x->convert_idx(X,x[1],x[2]),enumerate(idx))
    X.value[new_index...] = value
end

function Base.length(X::GP)
    return length(X.value)
end

In [4]:
GU = GamsUniverse()

@GamsSet(GU,:i,"Test",begin
    a, ""
    b, ""
end)

X = GP(GU,(:i,:i),"This is a test")

X[[:a],[:a]] = 1
X[[:a],[:b]] = 2
X[[:b],[:a]] = 3
X[[:b],[:b]] = 4

X

Description: This is a test
Domain: (:i, :i)

[1.0 2.0; 3.0 4.0]

In [5]:
c = findall(X.>2)
X[c]

2-element Vector{Float64}:
 3.0
 4.0

In [6]:
X[[:a],[:a]]

1.0

In [9]:
X[findall(X .>2)]=[10,11]

2-element Vector{Int64}:
 10
 11

In [10]:
X

Description: This is a test
Domain: (:i, :i)

[1.0 2.0; 10.0 11.0]

In [ ]:
findall(x->x>0,Y)

In [13]:
X[:i,:i] = X[:i,:i].+X[:i,:i]

2×2 Matrix{Float64}:
  2.0   4.0
 20.0  22.0

In [23]:
for i∈GU[:i]
    println("$i -> $(sum(X[:i,[i]]))") #Careful here
end

a -> 22.0
b -> 26.0


In [27]:
X[:i,:i] + X[:i,:i]

2×2 Matrix{Float64}:
  4.0   8.0
 40.0  44.0

In [29]:
for i∈GU[:i],j∈GU[:i]
    println(X[[i],[j]])
end

2.0
4.0
20.0
22.0


In [31]:
string(X.value)

"[2.0 4.0; 20.0 22.0]"

In [32]:
X.value

2×2 Matrix{Float64}:
  2.0   4.0
 20.0  22.0